In [608]:
import Derivative_Trading_Strategy.Utilities.Utilities_Resources

In [612]:
def retrieve_aggregate_bars_options(ticker, time_multiplier="1", timespan="minute", start_date="2023-01-09",
                                    end_date=datetime.datetime.today().strftime("%Y-%m-%d"), adjusted="True",
                                    sort="asc", limit=1200):

    url = f"https://api.polygon.io/v2/aggs/ticker/O:{ticker}/range/{time_multiplier}/{timespan}/{start_date}/{end_date}?adjusted={adjusted}&sort={sort}&limit={limit}&apiKey={apiKey}"

    response = requests.request("GET", url, headers=headers, data=payload).json()
    for i in range(0, len(response['results'])):
        response['results'][i]['t'] = pd.to_datetime(response['results'][i]['t'], unit='ms')
        response['results'][i]['Time'] = response['results'][i].pop('t')
        response['results'][i]['Volume_Weighted'] = response['results'][i].pop('vw')
        response['results'][i]['Open_Price'] = response['results'][i].pop('o')
        response['results'][i]['Lowest_Price'] = response['results'][i].pop('l')
        response['results'][i]['Highest_Price'] = response['results'][i].pop('h')
        response['results'][i]['Close_Price'] = response['results'][i].pop('c')
        response['results'][i]['Volume'] = response['results'][i].pop('v')
        response['results'][i]['Lot_Size'] = response['results'][i].pop('n')

    return response

#https://api.polygon.io/v2/aggs/ticker/O:SPY251219C00650000/range/1/day/2023-01-09/2023-01-09?adjusted=true&sort=asc&limit=120&apiKey=oIpqVdgJYK9nldqQ5j8JiPXxVVZptz0a

retrieve_aggregate_bars_options('AAPL')

KeyError: 'results'

In [558]:
def graph_option_history(data_retrieved):
    dataframe_of_values = pd.DataFrame(data_retrieved['results'])
    fig = px.line(dataframe_of_values, x="Time", y="Volume_Weighted", title=data_retrieved['ticker'])
    fig.show()


graph_option_history(a_2)

graph_option_history(a_3)

In [309]:
def generate_option_ticker(ticker):
    ticker_list = yf.Ticker(ticker)
    master_dict = {"Calls": {}, "Puts": {}}

    for i in range(0, len(ticker_list.options)):
        master_dict["Calls"][ticker_list.options[i]] = {}
        master_dict["Puts"][ticker_list.options[i]] = {}

    for j in master_dict["Calls"].keys():

        call_df_for_date = ticker_list.option_chain(date=j).calls
        puts_df_for_date = ticker_list.option_chain(date=j).puts

        for k in range(0, len(call_df_for_date['strike'])):
            master_dict["Calls"][j][call_df_for_date['strike'][k]] = call_df_for_date['contractSymbol'][k]
        for k in range(0, len(puts_df_for_date['strike'])):
            master_dict["Puts"][j][puts_df_for_date['strike'][k]] = puts_df_for_date['contractSymbol'][k]

    return master_dict

In [408]:
tsly.get_dividends().index[0]

Timestamp('2023-01-10 00:00:00-0500', tz='America/New_York')

In [459]:
def get_the_prior_and_posterior_n_day_periods_dividends(ticker, time_delta):
    temp_tick = yf.Ticker(ticker)
    dividend_dates = temp_tick.get_dividends().index
    new_list_of_dates = list(dividend_dates)
    first_and_last_values = []

    for dates in dividend_dates:
        time_delta_i = -1 * time_delta
        first_and_last_values.append(
            [dates + timedelta(days=time_delta_i, hours=0), dates - timedelta(days=time_delta_i, hours=0)])
        while time_delta_i < time_delta:
            new_list_of_dates.append(dates + timedelta(days=time_delta_i, hours=0))
            time_delta_i += 1

    temp_date_list = []
    for dates in new_list_of_dates:
        temp_date_list.append(datetime.datetime.strftime(dates, "%Y%m%d"))
    temp_date_list.sort()

    final_temp_list = []
    for date in temp_date_list:
        final_temp_list.append(datetime.datetime.strptime(date, "%Y%m%d").strftime("%Y-%m-%d"))

    return [final_temp_list, first_and_last_values]


m = get_the_prior_and_posterior_n_day_periods_dividends('TSLY', 5)


In [550]:
def graph_consecutive_time_plots(ticker, list_of_values):
    for pairs in list_of_values:

        date_1 = datetime.datetime.strftime(pairs[0], "%Y-%m-%d")
        date_2 = datetime.datetime.strftime(pairs[1], "%Y-%m-%d")
        try:
            result_1 = retrieve_aggregate_bars_stocks(ticker=ticker, start_date=date_1, end_date=date_2, timespan='day')
            graph_option_history(result_1)
        except KeyError:
            print('')


In [552]:
graph_consecutive_time_plots('TSLY', get_the_prior_and_posterior_n_day_periods_dividends('TSLY', 10)[1])